## 3.1. Linear Regression

*Studying and coding along with the printed book __„Dive into Deep Learning“__ by Aston Zhang, Zachary C. Lipton, Mu Li & Alexander J. Smola. The accompanying website for the chapter Linear Neural Networks for Regression > Linear Regression can be found at [d2l.ai](https://d2l.ai/chapter_linear-regression/linear-regression.html).*

__Solving a regression problem very much means predicting a numerical value.__

Common examples:

- Predicting prices of homes or stocks
- Predicting the length of stays for patients in a hospital
- Forecasting demand for retail sales

There are also prediction problems that are classification problems with the goal of predicting membership among a set of categories.

__Let's use the following scenario as an example:__ we want to estimate the *prices of houses* (in US\\$) based on their *size/area* in square feet and their *age* in years. 

Our goal is to __develop a model for predicting house prices__.

- The first step is to generate a *dataset* with sales prices, area and age for a number of homes.
- This dataset will be our *training dataset* or ***training set***
- Each row of the dataset is called an *example* (or *data point*, *instance*, *sample*).
- An example contais the data corresponding to one sale.
- What we're trying to predict is the sales price which in technical terminology is called a *label* (or ***target***).
- Variables (in our case age and area) upon which the predictions are based are called ***features*** (or *covariates*).

In [1]:
%matplotlib inline
import math
import time
import numpy as np
import torch

### 3.1.1. Basics

Some basic assumptions we make in *linear regression*:

1. We assume that the relationship between features $\mathbf{x}$ and target $y$ is approximately linear
   - The conditional mean $E[Y \mid X=\mathbf{x}]$ can be expressed as a weighted sum of the features $\mathbf{x}$
   - The target value might deviate from its expected value on account of observation noise
   - The assumption that any such noise is well behaved, following a Gaussian distribution
2. $n$ will be used to denote the number of examples in our dataset
   - Superscripts will be used to enumerate samples and targets: $\mathbf{x}^{(i)}$ denotes the $i^{\textrm{th}}$ sample
   - Subscripts will be used to index coordinates: $x_j^{(i)}$ denotes its $j^{\textrm{th}}$ coordinate

*(These notes are taken form the printed book and the GitHup repo [d2l-ai/d2l-en](https://github.com/d2l-ai/d2l-en); mathematical notation is copied from the github repo as I still have to figure out how to write and read it correctly)*

#### 3.1.1.1. Model

The description of how features can be transformed into an estimate of the target is what we call a *model*.

We assume the this transformation is linear. This assumption of linearity means that
- the expected value of the *target* (price) can be expressed as a weighted sum of the *features* (area and age):
  $$\textrm{price} = w_{\textrm{area}} \cdot \textrm{area} + w_{\textrm{age}} \cdot \textrm{age} + b.$$

- What we have here is an *affine transformation* of input features: a *linear transformation* of features via a weighted sum ***combined with a *translation* via the added bias***
  
- In this example $w_{\textrm{area}}$ and $w_{\textrm{age}}$ are the *weights* (***they determine the influence of each feature on our prediction***)
- $b$ is what we call a *bias* (other terms for it are *offset* or *intercept*) (***it determines the value of the estimate when all features are zero***)
  - the bias allows us to express all linear functions of our features

***Our goal for a given sample is*** 
- to choose the weights $\mathbf{w}$ and
- to choose the bias $b$
in a way that ***the predictions our model will make will fit the true prices observed as closely as possible***.

***Linear algebra notation for high-dimensional datasets that are commonly used in machine learning:***
$$\hat{y} = w_1  x_1 + \cdots + w_d  x_d + b$$

- Given an input of $d$ features
- each gets assigned an index (between $1$ and $d$)
- and our prediction is denoted with the "hat" symbol: $\hat{y}$

All features are collected into a vector $\mathbf{x} \in \mathbb{R}^d$ and all weights into a vector $\mathbf{w} \in \mathbb{R}^d$. By doing this ***our model can be expressed via the dot product*** between $\mathbf{w}$ and $\mathbf{x}$:
$$\hat{y} = \mathbf{w}^\top \mathbf{x} + b.$$
In this equation the vector $\mathbf{x}$ corresponds to the features of a single example.

If we want to refer to features of our entire dataset of $n$ examples we use the *design matrix* $\mathbf{X} \in \mathbb{R}^{n \times d}$ ($\mathbf{X}$ contains one row for every example and one column for every feature).

For a collection of features $\mathbf{X}$, the predictions $\hat{\mathbf{y}} \in \mathbb{R}^n$ can be expressed via the 

The following matrix-vector product expresses 
- the predictions $\hat{\mathbf{y}} \in \mathbb{R}^n$
- for a collection of features $\mathbf{X}$

$${\hat{\mathbf{y}}} = \mathbf{X} \mathbf{w} + b,$$

*Broadcasting is applied during the summation of the features*.

**Let's assume we've training dataset $\mathbf{X}$ and corresponding labels $\mathbf{y}$ which are known.**
- The goal of linear regression now is to find the weight vector $\mathbf{w}$ and the bias term $b$
- in a way that for given features of a new data example sampled from the same distribution as $\mathbf{X}$
- this new dataset's labels will be predicted with the smallest error

***It's important to be aware of that even when we are confident that the underlying relationship is linear, there might be a small amount of measurement error***
- Therefor we will incorporate a noise term to account for such errors

Two ***prerequisites*** we need to take into account are
1. a measure of the quality of some given model
2. a procedure for updating the model to improve its quality

Once we have solved this, we can go and search for the best *model parameters* $\mathbf{w}$ and $b$.

### 3.1.2. Vectorization for Speed

When training our models, we typically want to process whole minibatches of examples simultaneously. 
Doing this efficiently requires that we vectorize the calculations and leverage fast linear algebra libraries rather than writing costly for-loops in Python.

To see why this matters so much, let’s consider two methods for adding vectors. To start, we instantiate two 10,000-dimensional vectors containing all 1s. In the first method, we loop over the vectors with a Python for-loop. In the second, we rely on a single call to +.

In [7]:
n = 1000
a = torch.ones(n)
b = torch.ones(n)
print(a.shape)
print(b.shape)

torch.Size([1000])
torch.Size([1000])


#### __Benchmarking the workloads__

In [10]:
# adding them using a for-loop, one coordinate at a time
c = torch.zeros(n)
t = time.time()
for i in range(n):
    c[i] = a[i] + b[i]
f'{time.time() - t:.5f} sec'

'0.12503 sec'

In [12]:
# vectorizing code, pushing the mathematics to the library
# computing the elementwise sum with the reloaded + operator
t = time.time()
d = a + b
f'{time.time() - t:.5f} sec'

'0.00100 sec'